In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
from dotenv import load_dotenv
import pandas as pd
from joblib import Parallel, delayed

load_dotenv("./.env")

client_id = os.environ.get("SPOTIFY_CLIENT_ID")
client_secret = os.environ.get("SPOTIFY_CLIENT_SECRET")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret), requests_timeout=45)
# genres = sp.recommendation_genre_seeds() # only gets most popular genres
genres = pd.read_csv("genres.csv").values.tolist()
genres = [genre[0] for genre in genres]

In [ ]:
def getRecommendation(genre, i):
    try:
        return sp.search(q=f'genre: {genre}', type="track", limit=50, offset=i*50)
    except: None

def getRecommendationByGenre(genre):
        _tracks = Parallel(
            n_jobs=50
        )(
            delayed(getRecommendation)(genre, i) for i in range(0, 10)
        )
        return _tracks      


tracks = Parallel(
    n_jobs=50
)(
    delayed(getRecommendationByGenre)(genre) for genre in genres
)


In [63]:
def isNone(x):
    if x is not None:
        return x

cleaned = list(map(isNone, tracks))
flattened = [b for b in cleaned for b in b]

In [65]:
tracks_objects = list(map(lambda x: x["tracks"]["items"] if x is not None else dict({ "id": "" }), flattened))
flattened2 = [b for b in tracks_objects for b in b]
ids = list(map(lambda y: y["id"] if type(y) is dict else '', flattened2))
len(ids)

60104

In [68]:
deduped = list(set(ids))
df = pd.DataFrame(deduped).drop_duplicates()
df2 = pd.read_csv("./tracks2.csv").drop_duplicates()
df3 = pd.read_csv("./tracks3.csv").drop_duplicates()
df4 = pd.read_csv("./tracks4.csv").drop_duplicates()
df5 = pd.concat([df, df2["0"], df3["0"], df4["0"]], axis=0, ignore_index=True).drop_duplicates()
len(df5)

122278